In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import numpy as np 
import glob 
import warnings 
from scipy.stats import norm
from IPython.display import display, HTML
import scipy
warnings.filterwarnings("ignore")
%matplotlib inline
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
sns.set()

In [2]:
leagues = ['EPL','La_Liga','Bundesliga','Serie_A','Ligue_1','RFPL']

In [3]:
driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")

In [4]:
def getTeamStats(leagues):
    #Create the receiving dataFrame
    dfOut= []
    #Go through every league
    for league in leagues:
        #Establish Connection
        driver.get('https://understat.com/league/'+league)
        #Get the relevant object
        base = driver.find_element_by_id('league-chemp')
        #Get Rows
        rows = base.find_elements_by_tag_name('tr')
        #Create a list to receive values
        values=[]
        #Go through every row
        for item in range(1,len(rows)):
            #Get elements from the row
            row = rows[item].find_elements_by_tag_name('td')
            #Add those elements to the values list
            values.append([row[i].text for i in range(1,len(row))])
            #Go through every value and append it to the dataframe value by value
            for j in range(0,len(values)):
                dfOut.append([league,values[j][0],values[j][1],values[j][2],values[j][3],values[j][4],values[j][5],values[j][6],
                             values[j][7],values[j][8],values[j][9],values[j][10]])
    return dfOut

#Run the function
dfOut = getTeamStats(leagues=leagues)
#Convert to dataframe
dfOut = pd.DataFrame(dfOut,columns=['league','team','matches','wins','draws','losses','goals_for','goals_against','points','xG','xGA','xPTS'])
#Drop duplicate rows created
dfOut = dfOut.drop_duplicates()

In [6]:
teams = dfOut['team'].unique()

In [ ]:
"""Function mines players stats Data"""

def getPlayerStats(teams):
    #Create empty list for upcoming data
    data = []
    driver = webdriver.Chrome(r"C:/chromedriver_win32/chromedriver.exe")
    #Go through every team and get player stats for every team
    for i in teams:
        driver.get('https://understat.com/team/'+i+'/2018')
        baseTable = driver.find_element_by_id('team-players')
        tableRows = baseTable.find_elements_by_tag_name('tr')
        for item in range(1,len(tableRows)):
            row     = tableRows[item].find_elements_by_tag_name('td')
            Team    = i
            name    = row[1].text
            pos     = row[2].text
            apps    = row[3].text
            Min     = row[4].text
            goals   = row[5].text
            assists = row[6].text
            sh90    = row[7].text
            kp90    = row[8].text
            xg      = row[9].text
            xa      = row[10].text
            xg90    = row[11].text
            xa90    = row[12].text
            #Append to lsit
            data.append([Team,name,pos,apps,Min,goals,assists,sh90,kp90,xg,xa,xg90,xa90])
    #Create DataFrame from List
    df = pd.DataFrame(data)
    df = df.rename(columns={0:'Team',1:"Name",2: "Position",3: "Apps", 4:"Min", 5:"Goals_Scored"
                    , 6:"Assists",7: "Sh90",8: "KP90",9: "xG",10: "xA"
                    , 11:"xG90",12: "xA90"})
    return df

df = getPlayerStats(teams)

#Split df to Summary player per team Stats and Player stats
df_teams = df[df.Name =='']
df_players = df[df.Name != '']

In [22]:
df_summary_players_per_team.to_csv('Summary_player_per_team.csv')
df_players.to_csv('Players_stats.csv')
dfOut.to_csv('Teams_stats.csv')